In [1]:
from glob import glob
from pathlib import Path
from src.adaface.face_alignment import align
from src.adaface.inference import load_pretrained_model, to_input

# architecture = 'ir_50'
# pretrained_path = '/home/s3-subtitle-download/pretrained_models/AdaFace/adaface_ir50_ms1mv2.ckpt'
# model = load_pretrained_model(architecture, pretrained_path)

# ONNX

### ONNX batch


In [2]:
from PIL import Image
import torch
import onnxruntime as ort
from src.adaface.main import AdaFace

# ONNX_PATH = 'adaface.onnx'
ONNX_PATH = "/home/s3-subtitle-download/pretrained_models/AdaFace/adaface.onnx"

adaface = AdaFace(ONNX_PATH)

is_face = False
paths = sorted(glob("_test/*.jpg"))

# is_face = True  # もともとfaceがクロップされた画像か（クロップされた画像の場合, 顔検出がうまくいかないことがある）
# paths = sorted(glob('_test/cropped_face/*.jpg'), key=lambda x: int(Path(x).stem))

cropped_face_images = []
for path in paths:
    if is_face:
        cropped_face_image = Image.open(path).convert("RGB")
    else:
        cropped_face_image = align.get_aligned_face(path)
    cropped_face_images.append(cropped_face_image)

features = adaface.extract_features(cropped_face_images)

names = [Path(path).stem for path in paths]
features = {name: torch.tensor(feature).unsqueeze(0) for name, feature in zip(names, features)}

# Checkpoint

### single image


In [3]:
# features = {}
# for path in glob('_test/*.jpg'):
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     feature, _ = model(bgr_input)
#     features[Path(path).stem] = feature

### multiple images, bath processing


In [4]:
# # multiple images, bath processing
# import torch
# paths = glob('_test/*.jpg')

# faces = []
# for path in paths:
#     aligned_rgb_img = align.get_aligned_face(path)
#     bgr_input = to_input(aligned_rgb_img)
#     faces.append(bgr_input)

# faces = torch.cat(faces, dim=0)
# features = model(faces)[0]

# names = [Path(path).stem for path in paths]
# features = {name: feature.unsqueeze(0) for name, feature in zip(names, features)}

# Calculate Similarity


In [5]:
# calculate similarity
import torch.nn.functional as F

for name1, feature1 in features.items():
    for name2, feature2 in features.items():
        print(f"{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}")

hasegawa, hasegawa	1.0000
hasegawa, hasegawa_60	0.0286
hasegawa, taka    	-0.0336
hasegawa, toshi   	-0.0363
hasegawa, watanabe	0.0641
hasegawa_60, hasegawa	0.0286
hasegawa_60, hasegawa_60	1.0000
hasegawa_60, taka    	0.0104
hasegawa_60, toshi   	-0.0299
hasegawa_60, watanabe	0.4554
taka    , hasegawa	-0.0336
taka    , hasegawa_60	0.0104
taka    , taka    	1.0000
taka    , toshi   	0.1356
taka    , watanabe	-0.0112
toshi   , hasegawa	-0.0363
toshi   , hasegawa_60	-0.0299
toshi   , taka    	0.1356
toshi   , toshi   	1.0000
toshi   , watanabe	-0.0475
watanabe, hasegawa	0.0641
watanabe, hasegawa_60	0.4554
watanabe, taka    	-0.0112
watanabe, toshi   	-0.0475
watanabe, watanabe	1.0000


# Comparison to FaceRecognition


In [6]:
# import torch
# import face_recognition
# fr_features = {}
# for path in glob('_test/*.jpg'):
#     img = face_recognition.load_image_file(path)
#     ft = face_recognition.face_encodings(img)
#     fr_features[Path(path).stem] = torch.tensor(ft)
# #
# for name1, feature1 in fr_features.items():
#     for name2, feature2 in fr_features.items():
#         print(f'{name1:8}, {name2:8}\t{F.cosine_similarity(feature1, feature2).item():.4f}')